In [ ]:
import os

import json
import pickle

import sys
sys.path.append("..")

import numpy as np
import pandas as pd

import tensorflow as tf
from pytorch_transformers import RobertaTokenizer

from src.eqt.datasets import create_test_dataset_for_prediction
from src.eqt.preprocess_data import preprocess_data
from src.eqt.model_qbert import QBERT


def get_label_source(row, class_tag):
    if row[class_tag] != '':
        return row['label_source']
    return 'QBERT'


mappings = {
    'type': {
        'Ask about antecedent':  0,
        'Ask about consequence': 1,
        'Ask for confirmation':  2,
        'Irony':                 3,
        'Negative rhetoric':     4,
        'Positive rhetoric':     5,
        'Request information':   6,
        'Suggest a reason':      7,
        'Suggest a solution':    8,
    },
    'intent': {
        'Amplify excitement': 0,
        'Amplify joy':        1,
        'Amplify pride':      2,
        'De-escalate':        3,
        'Express concern':    4,
        'Express interest':   5,
        'Moralize speaker':   6,
        'Motivate':           7,
        'Offer relief':       8,
        'Pass judgement':     9,
        'Support':           10,
        'Sympathize':        11,
    }
}

class_type = "type"
peak_lr = 2e-5
checkpoints_path = ""

In [1]:
# Load dataset


NameError: name 'RobertaTokenizer' is not defined

In [ ]:
# Load model checkpoint
num_layers         = 12
d_model            = 768
num_heads          = 12
dff                = d_model * 4
hidden_act         = "gelu"
dropout_rate       = 0.1
layer_norm_eps     = 1e-5
max_position_embed = 514

tokenizer  = RobertaTokenizer.from_pretrained("roberta-base")
vocab_size = tokenizer.vocab_size

lab_mapping  = mappings[class_type]
pred_mapping = {v: k for k, v in lab_mapping.items()}
num_classes  = len(pred_mapping.keys())

adam_beta_1  = 0.9
adam_beta_2  = 0.98
adam_epsilon = 1e-6

qbert = QBERT(num_layers, 
              d_model, 
              num_heads, 
              dff, 
              hidden_act, 
              dropout_rate,
              layer_norm_eps, 
              max_position_embed, 
              vocab_size, 
              num_classes)
optimizer = tf.keras.optimizers.Adam(peak_lr, 
                                     beta_1=adam_beta_1, 
                                     beta_2=adam_beta_2,
                                     epsilon=adam_epsilon)
ckpt = tf.train.Checkpoint(model=qbert, optimizer=optimizer)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoints_path, max_to_keep=None)